# Using Sagemaker Processing for post-processing aggregation

There are situations when you need to perform post-processing on your batch predictions. A good example of such use case could be to calculate aggregate statistics, e.g. in case of Named Entity Recoknition task calculate how many times specific tokens occur in prediction dataset. 

Amazon Sagemaker allows you to easily implement this type of scenarios using [Sagemaker Processing feature](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html). In this notebook, we'll explore how to calculate number of total occurences for specific tokens using Sagemaker Processing.

With Amazon SageMaker Processing jobs, you can leverage a simplified, managed experience to run data pre- or post-processing and model evaluation workloads on the Amazon SageMaker platform. A processing job downloads input from Amazon Simple Storage Service (Amazon S3), then uploads outputs to Amazon S3 during or after the processing job.


![alt text](./images/proc_1.jpg)

## Building custom processing container

For this exercise we will use a simple processing container built from scratch. This will give some idea how easy it is to port your existing processing code/pipeline and run it using Sagemaker capabilities.

Let's define parameteres of our customer container, then build and push it to your account ECR.

In [44]:
import boto3 
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()

ACCOUNT_ID = sess.boto_session.client('sts').get_caller_identity()['Account']
CONTAINER_NAME="custom-processing"
TAG = "latest"
REGION = "us-east-2"

In [45]:
# login to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin {ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [75]:
! pygmentize -l docker Dockerfile.postproc

FROM python:3.7-slim-buster

########### Installing packages ##########
RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3
ENV PYTHONUNBUFFERED=TRUE

########### Configure processing scripts ##########
ARG code_dir=/opt/ml/code
RUN mkdir -p $code_dir
COPY processing_sources $code_dir
WORKDIR $code_dir

ENTRYPOINT ["python3","processing.py"]


Now, let's build container and push it to ECR

In [76]:
! ./build_and_push.sh $CONTAINER_NAME $TAG Dockerfile.postproc

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  197.6kB
Step 1/8 : FROM python:3.7-slim-buster
 ---> 87b1022604d5
Step 2/8 : RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3
 ---> Using cache
 ---> a5a5041b02a0
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 0fd4d5fe7349
Step 4/8 : ARG code_dir=/opt/ml/code
 ---> Using cache
 ---> 51db599d8da2
Step 5/8 : RUN mkdir -p $code_dir
 ---> Using cache
 ---> 17de15081774
Step 6/8 : COPY processing_sources $code_dir
 ---> 0845b73cf427
Step 7/8 : WORKDIR $code_dir
 ---> Running in 683844f39f11
Removing intermediate container 683844f39f11
 ---> ab7483d0f276
Step 8/8 : ENTRYPOINT ["python3","processing.py"]
 ---> Running in a1d24

In [21]:
IMAGE_URI = f"{ACCOUNT_ID}.dkr.ecr.us-east-2.amazonaws.com/{CONTAINER_NAME}:{TAG}"
print("Following container will be used for processing job: ", IMAGE_URI)

Following container will be used for processing job:  553020858742.dkr.ecr.us-east-2.amazonaws.com/custom-processing:latest


## Preparing processing script



In [77]:
! pygmentize processing_sources/processing.py

from argparse import ArgumentParser
import os
import json

def main(args):
    
    tokens = args.tokens.split(',')
    
    # initiate dict to store token counts results
    token_counts = {token : 0 for token in tokens}
    
    files = []
    for _,_,filenames in os.walk(args.input_dir):
        files.extend(filenames)
    
    for file in files:
        
        with open(file, 'r') as file:
            file_string = file.read().replace('\n','')
        
        for k, v in token_counts.items():
            token_counts[k] += file_string.count(k)
    
    
    with open(os.path.join(args.output_dir, 'token_counts.json'), 'w') as file:
        json.dump(token_counts, file)


if __name__ == "__main__":
    
    # Parse common arguments
    print('Starting training...')
    parser = ArgumentParser()
    parser.add_argument('--tokens', type=str, required=True, help="specify which tokens to count in input files")
    parser.add_argument('--input-dir', type=str, default="/opt/ml/processi

## Run Sagemaker Processing job



In [82]:
bucket = "vadimd-batch-transform"

prefix_input = 'flair-output' 
prefix_output = 'postproc-output' 

input_s3_path = f"s3://{bucket}/{prefix_input}"    # S3 path where results of our inference job are stored
output_s3_path = f"s3://{bucket}/{prefix_output}"  # S3 path where we'll upload aggregated processing results

Define which tokens we'd like to count as part of processing. This dictionary will be passed as command line arguments to our processing script.

In [78]:
tokens_to_count = ["<B-LOC>", "<E-LOC>", "<I-LOC>", "<S-LOC>"]

In [79]:
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

processor = Processor(image_uri=IMAGE_URI,
                      role=role,
                      instance_count=1,
                      base_job_name="nlp-processing-2",
                      sagemaker_session=sess, 
                      instance_type="ml.m5.xlarge")
                     
processor.run(inputs=[ProcessingInput(source=input_s3_path,
                                      destination='/opt/ml/processing/input_data')],
                                      outputs=[ProcessingOutput(source='/opt/ml/processing/processed_data',
                                      destination=output_s3_path)],
                                      arguments=["--tokens", ','.join(tokens_to_count)])


Job Name:  nlp-processing-2-2020-06-02-00-00-26-689
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://vadimd-batch-transform/flair-output', 'LocalPath': '/opt/ml/processing/input_data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'S3Output': {'S3Uri': 's3://vadimd-batch-transform/postproc-output', 'LocalPath': '/opt/ml/processing/processed_data', 'S3UploadMode': 'EndOfJob'}}]
...................Starting training...



In [83]:
! aws s3 cp s3://{output_s3_path}/token_counts.json token_counts.json
! cat token_counts.json

fatal error: Parameter validation failed:
Invalid bucket name "s3:": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:s3:[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$"
cat: token_counts.json: No such file or directory
